In [1]:
#going to want create an instance of the the Dekel profile and give it the final state parameters so it can 
#just call rho(radius) etc and plot 
import numpy as np
import astropy.units as u
import pandas as pd
import astropy.constants as const
import matplotlib as mpl
import matplotlib.pyplot as plt
from glob import iglob
import glob
import random

/home/dropulic/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
from scipy.signal import argrelmin, argrelextrema, argrelmax
import numpy.ma as ma

In [3]:
import sys
sys.path.insert(0, '/tigress/dropulic/SatGen/')
import aux
import orbit as orb
from scipy.interpolate import interp1d
from profiles import Dekel, MN, EnergyAngMomGivenRpRa, Phi
from galhalo import Reff
#mass, DekelConcentration, DekelSlope, VirialOverdensity
#halo = Dekel(117781540.8609773, 3.0486991 ,1.4538499, 227.27994,z=0.)
import os
#import astropy
#from astropy.cosmology import FlatLambdaCDM
#cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
#import astropy.units as u
#import astropy.cosmology.units as cu
#u.add_enabled_units(cu)

>>> Normalizing primordial power spectrum P(k)=(k/k_0)^n_s ...
    such that sigma(R=8Mpc/h) =   0.8000.
>>> Tabulating sigma(M,z=0) ...
    From now on, sigma(M,z) is computed by interpolation.
>>> Tabulating z(W) and z(t_lkbk)...
>>> Preparing output redshifts for merger trees ...
    Number of output redshifts =  354, up to z = 20.06
>>> Tabulating Parkinson+08 J(u_res) ...
>>> Building interpolation grid for Green+19 M(<r|f_b,c)...
>>> Building interpolation grid for Green+19 sigma(r|f_b,c)...
>>> Building interpolation grid for Green+19 d2Phidr2(r|f_b,c)...
>>> Building interpolator for Jiang+15 orbit sampler...


In [4]:
import load_galaxy

In [5]:
galaxy = load_galaxy.Galaxy()

/scratch/gpfs/dropulic/power_spectrum/SatGen_trees/SatGen/OUTPUT_SAT_MW_fd0.05_fb0.00_NIHAO_SatEvo_Mres3/tree769_lgM12.38.npz
flattening 25.0
disk fraction 0.05
stellar_mass_floor 500000.0
(10670,)
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


In [6]:
print("Attributes of the Galaxy object:", list(vars(galaxy).keys()))

Attributes of the Galaxy object: ['flattening', 'fd', 'stellar_mass_floor', 'host_coords_alltime_list', 'idx_zaccs', 'pericenter_locs', 'apocenter_locs', 'all_apo', 'all_peri', 'coordinates_peri', 'coordinates_apo', 'velocities_apo', 'velocities_peri', 'coordinates_hold', 'velocities_hold', 'velocity_dispersion', 'sat_disk_rhobar', 'sat_stellar_disk_rhobar', 's001s', 'rhs', 'rh_halfs', 'rho_bars', 'mass_ratios', 'ras', 'rps', 'eccs', 'mass_lost_alls', 'stellar_mass_lost_alls', 'sat_distances', 'coordinates_SG', 'CosmicTime', 'host_Mvir', 'order_variable_SG', 'parentid_SG', 'host_mstar', 'zacc_idx_SG', 'host_coord', 'host_Rvir', 'redshift', 'host_conc', 'host_at_Mvir', 'host_at_dconc', 'host_at_slope', 'host_at_vovd', 'host_at_rvir', 'host_at_mstar', 'order3_locs_SG', 'parent_zacc_SG', 'host_coords_dekel', 'host_coords_alltime', 'array_s', 'vels_array_s', 'z', 'x', 'y', 'sat_final_mvir', 'sat_final_rvir', 'sat_final_mstar', 'Rvirs', 'Dekel_conc_s', 'Dekel_slope_s', 'Dekel_virialovd_s', 

In [7]:

mapped_attributes = [
    'mass_lost_alls',  # np.log10(mass_lost_all[sat_i])
    'rps',  # np.log10(peri[sat_i])
    'rh_halfs',  # np.log10(R_half[sat_i]*1000)
    'mass_ratios',  # np.log10(M_star_over_halo[sat_i])
    'rho_bars',  # np.log10(rho_half[sat_i]+sat_stellar_disk_rhobar_half[sat_i])
    'ras',  # apo[sat_i]
    'eccs',  # ecc[sat_i]
    'sat_final_mstar',  # np.log10(M_star[sat_i].round(0))
    'sat_final_mvir',  # np.log10(M_halo[sat_i].round(0))
    'sat_final_rvir',  # R_vir[sat_i]
    'stellar_mass_lost_alls',  # np.log10(stellar_mass_lost_all[sat_i])
    'CosmicTime',  # cosmo.age(redshift[idx_zaccreted[sat_i]]*cu.redshift)/u.Gyr
    'allz_mstar_s',  # allz_mstar[sat_i,:]
    'allz_m_s',  # allz_m[sat_i,:]
    'sat_distances',  # sat_distances[sat_i]
    'pericenter_locs',  # pericenter_locs[sat_i]
    'apocenter_locs',  # apocenter_locs[sat_i]
    'CosmicTime',  # np.diff(cosmo.age(redshift[apocenter_locs[sat_i]])/u.Gyr)
    'coordinates_apo',  # coord_apo[sat_i]
    'coordinates_peri',  # coord_peri[sat_i]
    'velocities_apo',  # vel_apo[sat_i]
    'all_apo',  # all_apo_list[sat_i]
    'all_peri',  # all_peri_list[sat_i]
    'CosmicTime',  # 14 - cosmo.age(redshift[idx_zaccreted[sat_i]]*cu.redshift)/u.Gyr
    'num_sat_s',  # host_number[sat_i]
    'allz_mstar_s',  # np.max(allz_mstar[sat_i,:], axis = 0)
    'z',  # coord[sat_i,:,2]
    'x',  # np.multiply(coord[sat_i,:,0], np.cos(coord[sat_i,:,1]))[:]
    'y',  # np.multiply(coord[sat_i,:,0], np.sin(coord[sat_i,:,1]))[:]
    'idx_zaccs',  # idx_zaccreted[sat_i]
    'num_sat_s',  # sat_nums[sat_i]
    'coordinates_hold',  # coord[sat_i]
    'velocity_dispersion',  # velocity_dispersion_z0[sat_i]
    'coordinates_hold',  # coord_arr[sat_i]
    'velocities_hold',  # vel_arr[sat_i]
    's001s',  # s001_val[sat_i]
    'allz_mstar_s',  # allz_mstar[sat_i,idx_zaccreted[sat_i]]
    'allz_m_s',  # allz_m[sat_i,idx_zaccreted[sat_i]]
    'velocities_peri',  # vel_peri[sat_i]
    'redshift'
]



In [11]:
import importlib
#importlib.reload(load_galaxy)
#galaxy = load_galaxy.Galaxy()
substructure_df = galaxy.get_filtered_rows_as_dataframe(mapped_attributes)

In [11]:
columns_to_log10 = [
    'mass_lost_alls', 
    'rps', 
    'rh_halfs', 
    'mass_ratios', 
    'rho_bars', 
    'sat_final_mstar', 
    'sat_final_mvir', 
    'stellar_mass_lost_alls'
]

# Apply np.log10 to the specified columns
substructure_df[columns_to_log10] = substructure_df[columns_to_log10].apply(np.log10)



TypeError: loop of ufunc does not support argument 0 of type numpy.ndarray which has no callable log10 method

In [ ]:
# Display the transformed DataFrame

print(substructure_df.iloc[0].all_peri)

In [9]:
import importlib
importlib.reload(substructure_z0)

<module 'substructure_z0' from '/scratch/gpfs/dropulic/StreamGen/StreamGen_package/substructure_z0.py'>

In [38]:
import substructure_z0
import MC_metric

importlib.reload(substructure_z0)
importlib.reload(MC_metric)
subs = substructure_z0.Substructure(substructure_df, 'trial', galaxy)
subs.main_processing_loop()

Directory already exists: /scratch/gpfs/dropulic/StreamGen/StreamGen_package/trial/
20
Loaded MC files
upper_bin_edge 15839.778694961213
bin_no 3
False
3.732047389553197e-05 2.5501894882237985e-05
in integrate_orbit
coord_at [[ 56.909016  20.951082  37.8145   280.6569   175.07861    6.523808]
 [ 39.284653  20.684752  35.805687 299.3686   253.64052   65.94126 ]
 [ 24.58806   20.033144  29.954817 141.18524  405.2891   134.42375 ]
 ...
 [  0.         0.         0.         0.         0.         0.      ]
 [  0.         0.         0.         0.         0.         0.      ]
 [  0.         0.         0.         0.         0.         0.      ]]
coord_at[redshift_id] [ 56.909016  20.951082  37.8145   280.6569   175.07861    6.523808]
None
period_est_correction 0
huber exception!
in stream_or_shell_integrate
deltaPsi 4.876545902130235
Tr 1.328
mu 0.2889663390142649
deltaPsi 4.875340288169861
Tr 1.323
mu 0.28890259348720404
deltaPsi 4.87476711531858
Tr 1.32
mu 0.28832984476767176
deltaPsi 4.87552